In [103]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [104]:
# Import datasets
Mov=pd.read_csv('movies.csv',sep=',')
Rat=pd.read_csv('ratings.csv')
Osc=pd.read_csv('the_oscar_award.csv')
## Now start cleaning
# Transforming movies dataset: extract the years
def ext_movname(tlist):
    lst=tlist.copy()
    year=tlist.copy()
    for i in range(len(tlist)):
        raw=tlist[i].rstrip()
        try:
            year[i]=int(raw[-5:-1])
            lst[i]=raw[:-7]
        except:
            year[i]=np.nan
            lst[i]=raw
    return [lst,year]
u=ext_movname(Mov['title'])
Mov['title']=u[0]
Mov['year']=u[1]
CMov=Mov[Mov.year>=1994]
CMov=CMov.reset_index(drop=True)
# Function for recorrect the film name
def recorr(string):
    if '(' in string:
        nstr=string.split(' (')[0]
    else:
        nstr=string
    if ',' not in nstr:
        return nstr
    else:
        lst=nstr.split(sep=', ')
        if lst[-1] not in {'The', 'A'}:
            return nstr
        else:
            nlst=lst[:-1]
            new=lst[-1]+' '
            for j in nlst:
                new=new+j
            return new
CMov['title']=CMov['title'].apply(recorr)
# Choose the ceremony year and file name columns
C_osc=Osc[['year_ceremony','film','year_film']].dropna().copy()
C_osc=C_osc.reset_index(drop=True)
C_osc['film']=C_osc['film'].apply(recorr)
for i in range(len(C_osc)):
    if C_osc['film'][i][-1]==';':
        u=C_osc['film'][i][-1]
        C_osc['film'][i]=u[:-1]
C_osc=C_osc[C_osc.year_ceremony>=1995]
C_osc=C_osc.drop_duplicates(subset=['film','year_film'], keep='first').reset_index(drop=True)

/Users/lihuayu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [105]:
D_osc={(C_osc['film'][i],C_osc['year_film'][i]):C_osc['year_ceremony'][i] for i in range(len(C_osc))}

lst=[]
for i in range(len(CMov['title'])):
    if (CMov['title'][i],CMov['year'][i]) in D_osc:
        lst.append(D_osc[(CMov['title'][i],CMov['year'][i])])
    else:
        lst.append(0)
CMov['Oscar']=lst  
cMov=CMov[CMov.Oscar!=0]

In [107]:
# Transfering timestamp: extract the year data
def extract_year(time_se):
    return time.localtime(time_se).tm_year
Ye=Rat['timestamp'].apply(extract_year)
Rat['timestamp']=Ye

In [110]:
# Merge CMov and Rat, and filter the available rating times
R_Mov=pd.merge(cMov,Rat,on=['movieId'])
Ava_rating=R_Mov.reset_index(drop=True)[['movieId','title','genres','Oscar','timestamp','rating']]

In [113]:
prev=Ava_rating[((Ava_rating.timestamp<=Ava_rating.Oscar) & 
                 (Ava_rating.timestamp>=Ava_rating.Oscar-1))][['movieId','genres','rating']]
after=Ava_rating[((Ava_rating.timestamp>Ava_rating.Oscar) & 
                  (Ava_rating.timestamp<=Ava_rating.Oscar+1))][['movieId','genres','rating']]

In [116]:
# Now calculate for the parameters
glist=['movieId','genres']
gprev=prev.groupby(glist,as_index=False)
Stat_prev=gprev.count()
meanp=gprev.mean()['rating']
Stat_prev['prev_mean']=meanp
varp=gprev.var().fillna(0)['rating']
Stat_prev['prev_var']=varp
Stat_prev=Stat_prev.drop(['rating'],axis=1)

In [118]:
# Now calculate for the parameters
gafter=after.groupby(glist,as_index=False)
Stat_after=gafter.count()
meana=gafter.mean()['rating']
Stat_after['after_mean']=meana
vara=gafter.var().fillna(0)['rating']
Stat_after['after_var']=vara
Stat_after=Stat_after.drop(['rating'],axis=1)

In [141]:
# Now merge
Compare=pd.merge(Stat_prev,Stat_after,how='left',on=['movieId','genres'])

In [142]:
Compare['mean_diff']=Compare['after_mean']-Compare['prev_mean']
Compare['var_diff']=Compare['after_var']-Compare['prev_var']

In [143]:
# Adding 9 genre columns
Sp=Compare.shape
Hot_mat=np.array([0]*Sp[0]*9)
Hot_mat.shape=(Sp[0],9)
Hot_mat
for i in range(Sp[0]):
    gen=Compare['genres'][i].split(sep='|')
    if 'Action' in gen:
        Hot_mat[i,0]=1
    if 'Adventure' in gen:
        Hot_mat[i,1]=1
    if 'Comedy' in gen:
        Hot_mat[i,2]=1
    if 'Crime' in gen:
        Hot_mat[i,3]=1
    if 'Drama' in gen:
        Hot_mat[i,4]=1
    if 'Fantasy' in gen:
        Hot_mat[i,5]=1
    if 'Romance' in gen:
        Hot_mat[i,6]=1
    if 'Sci-Fi' in gen:
        Hot_mat[i,7]=1
    if 'Thriller' in gen:
        Hot_mat[i,8]=1
Compare['is_Action']=Hot_mat[:,0]
Compare['is_Adventure']=Hot_mat[:,1]
Compare['is_Comedy']=Hot_mat[:,2]
Compare['is_Crime']=Hot_mat[:,3]
Compare['is_Drama']=Hot_mat[:,4]
Compare['is_Fantasy']=Hot_mat[:,5]
Compare['is_Romance']=Hot_mat[:,6]
Compare['is_Sci-Fi']=Hot_mat[:,7]
Compare['is_Thriller']=Hot_mat[:,8]

In [144]:
Compare=Compare.drop(['genres'],axis=1)

In [147]:
Compare.to_csv('New_Comp_change3.csv')

In [154]:
# Another file
NMov=CMov[CMov.Oscar==0]
ExMov=pd.merge(NMov,Rat,on=['movieId'])
Ext_rating=ExMov.reset_index(drop=True)

In [158]:
E_prev=Ext_rating[(Ext_rating.timestamp<=Ext_rating.year+1)][['movieId','genres','rating']]
E_after=Ext_rating[(Ext_rating.timestamp>Ext_rating.year+1) & (Ext_rating.timestamp<=Ext_rating.year+2)][['movieId','genres','rating']]
min_comment=10

In [162]:
# Now calculate for the parameters
E_gprev=E_prev.groupby(glist,as_index=False)
Stat_Eprev=E_gprev.count()
E_meanp=E_gprev.mean()['rating']
Stat_Eprev['prev_mean']=E_meanp
E_varp=E_gprev.var().fillna(0)['rating']
Stat_Eprev['prev_var']=E_varp
Stat_Eprev=Stat_Eprev[Stat_Eprev.rating>=min_comment]
Stat_Eprev=Stat_Eprev[['movieId','genres','prev_mean','prev_var']]

In [163]:
# Now calculate for the parameters
E_gafter=E_after.groupby(glist,as_index=False)
Stat_Eafter=E_gafter.count()
E_meana=E_gafter.mean()['rating']
Stat_Eafter['after_mean']=E_meana
E_vara=E_gafter.var().fillna(0)['rating']
Stat_Eafter['after_var']=E_vara
Stat_Eafter=Stat_Eafter[Stat_Eafter.rating>=min_comment]
Stat_Eafter=Stat_Eafter[['movieId','genres','after_mean','after_var']]

In [166]:
# Now merge
ECompare=pd.merge(Stat_Eprev,Stat_Eafter,how='inner',on=['movieId','genres'])
ECompare['mean_diff']=ECompare['after_mean']-ECompare['prev_mean']
ECompare['var_diff']=ECompare['after_var']-ECompare['prev_var']

In [168]:
# Adding 9 genre columns
Sp=ECompare.shape
Hot_mat=np.array([0]*Sp[0]*9)
Hot_mat.shape=(Sp[0],9)
Hot_mat
for i in range(Sp[0]):
    gen=ECompare['genres'][i].split(sep='|')
    if 'Action' in gen:
        Hot_mat[i,0]=1
    if 'Adventure' in gen:
        Hot_mat[i,1]=1
    if 'Comedy' in gen:
        Hot_mat[i,2]=1
    if 'Crime' in gen:
        Hot_mat[i,3]=1
    if 'Drama' in gen:
        Hot_mat[i,4]=1
    if 'Fantasy' in gen:
        Hot_mat[i,5]=1
    if 'Romance' in gen:
        Hot_mat[i,6]=1
    if 'Sci-Fi' in gen:
        Hot_mat[i,7]=1
    if 'Thriller' in gen:
        Hot_mat[i,8]=1
ECompare['is_Action']=Hot_mat[:,0]
ECompare['is_Adventure']=Hot_mat[:,1]
ECompare['is_Comedy']=Hot_mat[:,2]
ECompare['is_Crime']=Hot_mat[:,3]
ECompare['is_Drama']=Hot_mat[:,4]
ECompare['is_Fantasy']=Hot_mat[:,5]
ECompare['is_Romance']=Hot_mat[:,6]
ECompare['is_Sci-Fi']=Hot_mat[:,7]
ECompare['is_Thriller']=Hot_mat[:,8]

In [170]:
ECompare=ECompare.drop(['genres'],axis=1)

In [172]:
ECompare.to_csv('New_Ex_Comp_change3.csv')